## 1. Instalação dos pacotes necessários

In [ ]:
%pip install requests
%pip install beautifulsoup4
%pip install pandas
%pip install numpy
%pip install scikit-learn
%pip install plotly

## 2. Script do WebScrapping

Código abaixo apenas para demonstração no notebook de como foi implementado.
Para executa-lo, é necessário rodar o arquivo main.py

No terminal com o local da aplicação selecionado, siga os seguintes passos:
1. pip install -r requirements.txt
2. python main.py

In [135]:
# # Tamanho máximo de resultados por pesquisa
# tamanho_resultado = 250
# # Lista de produtos para pesquisar
# pesquisa = [ 'rtx 4060 ti', 'rtx 4070 ti', 'rtx 4080',
#             'rx 7800 xt', 'rx 7700 xt', 'rx 7600 xt' ]

# # Loop principal
# while True:
#     try:
#         print(f'Iniciando scrapping em [{len(pesquisa)}] produtos.')
        
#         # Itera sobre cada item na lista de pesquisa
#         for x in pesquisa:
#             print(f'Scrapping no produto [{x}] iniciado!')
#             # URL da pesquisa para o produto atual
#             url = f'https://www.kabum.com.br/busca/{x}?page_number=1&page_size={tamanho_resultado}&facet_filters=eyJjYXRlZ29yeSI6WyJIYXJkd2FyZSJdfQ==&sort=most_searched&variant=catalog'
#             # Obtém os preços dos produtos na página de pesquisa
#             products = get_price(url)
#             # Lista para armazenar informações dos produtos encontrados
#             products_info = []
#             print(f'Foi(ram) encontrado(s) [{len(products)}] anuncios para o produto [{x.upper()}]')
#             # Itera sobre cada produto encontrado
#             for product in products:
#                 # Verifica se o link do produto contém o nome do produto na pesquisa
#                 if x.replace(' ', '-') in product['link']:
#                     print(product)
#                     # Obtém informações detalhadas do produto
#                     info = get_product_info(url, product, x)
#                     if info:
#                         products_info.append(info)

#             print(f'Scrapping no produto [{x.upper()}] finalizado!')
#             # Registra o número de produtos encontrados no log
#             save_log(Status.INFO.name, f'Quantidade de produto(s) encontrado(s) para {x.upper()}: {len(products_info)}')

#             list_products = []
#             # Se houver informações de produtos, adiciona à lista
#             if products_info:
#                 for pi in products_info:
#                     list_products.append(pi)
                    
#                 print(f'Salvando dados coletados do produto [{x.upper()}]!')
#                 # Salva os dados coletados em um arquivo
#                 save_to_file(list_products)
#                 print(f'Dados coletados do produto [{x.upper()}] salvo com sucesso!')

#         print(f'Scrapping finalizado!')
#         # Registra no log o sucesso na coleta de dados
#         save_log(Status.SUCCESS.name, f'{len(list_products)} dados coletados')
#     except Exception as e:
#         print(f"Erro ao obter preços dos produtos na Kabum: {e}")
#         # Registra no log o erro ocorrido
#         save_log(Status.ERRO.name, e)

## 3. Importação das Bibliotecas Utilizadas

In [165]:
import datetime
import numpy as np
import pandas as pd
from plotly import *
import plotly.graph_objs as go
import plotly.express as px
import re

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, Normalizer
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV, learning_curve
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.datasets import make_regression

## 4. Carregando os Dados

In [166]:
# Carrega os dados coletados do arquivo CSV
df = pd.read_csv("../data/scraped_data.csv", sep=";")

# Exibe as primeiras linhas do dataframe
df.head()

,full_name,model,link,sku,brand_name,weight,price,primePrice,primePriceWithDiscount,oldPrice,oldPrimePrice,priceWithDiscount,discountPercentage,rating,ratingCount,available,warranty,date
0,placa-de-video-rtx-4070-ti-ventus-msi-nvidia-g...,rtx 4070 ti,https://www.kabum.com.br//produto/403373/placa...,403373,MSI,1732,6117.64,5934.11,5043.99,6117.64,6117.64,5199.99,15,4,8,True,36 meses de garantia,2024-04-25 03:36:29.337716
1,placa-de-video-rtx-4070-ti-super-gigabyte-gami...,rtx 4070 ti,https://www.kabum.com.br//produto/517737/placa...,517737,Gigabyte,1816,6941.16,0.00,0.00,0.00,0.00,5899.99,15,0,0,True,36 meses de garantia,2024-04-25 03:36:29.337716
2,placa-de-video-rtx-4070-ti-super-msi-16g-ventu...,rtx 4070 ti,https://www.kabum.com.br//produto/520532/placa...,520532,MSI,1198,6352.93,0.00,0.00,0.00,0.00,5399.99,15,5,2,True,12 meses de garantia,2024-04-25 03:36:29.337716
3,placa-de-video-rtx-4070-ti-gaming-x-slim-12g-m...,rtx 4070 ti,https://www.kabum.com.br//produto/495568/placa...,495568,MSI,1890,6117.64,5934.11,5043.99,6117.64,6117.64,5199.99,15,5,9,True,12 meses de garantia,2024-04-25 03:36:29.337716
4,placa-de-video-rtx-4070-ti-super-gaming-verto-...,rtx 4070 ti,https://www.kabum.com.br//produto/520758/placa...,520758,Pny,1570,5199.99,0.00,0.00,0.00,0.00,5199.99,0,0,0,True,36 meses de garantia,2024-04-25 03:36:29.337716


## 5. Análise Exploratória

### 5.1. Tamanho do DataFrame

In [167]:
# Verifica o tamanho do dataframe
print(f'O dataframe possui {len(df)} linhas e {len(df.columns)} colunas.')

O dataframe possui 9634 linhas e 18 colunas.


### 5.2. Tipos de Dados

In [168]:
# Verifica os tipos de dados das colunas
print('\nTipos de dados das colunas:')
print(df.dtypes)


Tipos de dados das colunas:
full_name                  object
model                      object
link                       object
sku                         int64
brand_name                 object
weight                      int64
price                     float64
primePrice                float64
primePriceWithDiscount    float64
oldPrice                  float64
oldPrimePrice             float64
priceWithDiscount         float64
discountPercentage          int64
rating                      int64
ratingCount                 int64
available                    bool
warranty                   object
date                       object
dtype: object


### 5.3. Verificação de Valores Nulos

In [169]:
# Verifica se há valores nulos no dataframe
print('\nValores nulos por coluna:')
print(df.isnull().sum())


Valores nulos por coluna:
full_name                    0
model                        0
link                         0
sku                          0
brand_name                   0
weight                       0
price                        0
primePrice                   0
primePriceWithDiscount       0
oldPrice                     0
oldPrimePrice                0
priceWithDiscount            0
discountPercentage           0
rating                       0
ratingCount                  0
available                    0
warranty                  4145
date                         0
dtype: int64


### 5.4. Métodos úteis

In [170]:
def extrair_numeros(texto):
    if pd.isna(texto):
        return 0
    else:
        numeros = re.findall(r'\d+', texto)
        if numeros:
            return int(numeros[0])
        else:
            return 0

### 5.5. Tratativas das Colunas

In [171]:
# Faz uma cópia do DataFrame original 'df' e atribui à variável 'df2'
df2 = df.copy()

# Remove espaços em branco no início e no fim de cada string na coluna 'full_name'
df2['full_name'] = df2['full_name'].str.strip()

# Converte a coluna 'available' para o tipo de dado inteiro
df2['available'] = df2['available'].astype(int)

# Verifica se os valores na coluna 'warranty' são do tipo string antes de chamar a função 'extrair_numeros'
# df2['warranty'] = df2['warranty'].apply(lambda x: extrair_numeros(str(x)))
df2['warranty'] = df2['warranty'].astype(str).apply(extrair_numeros)

# Converte a coluna 'date' para o tipo de dado datetime
df2['date'] = pd.to_datetime(df2['date'])

# Converte os valores da coluna 'model' para letras maiúsculas
df2['model'] = df2['model'].str.upper()

In [172]:
# Cria uma nova Series 'df3' contendo apenas os valores da coluna 'full_name' do DataFrame 'df2'
df3 = df2['full_name']

# Remove a palavra 'Placa' de cada string na Series 'df3'
df3 = df3.str.replace('Placa', '')

# Remove as variações de 'de vídeo' de cada string na Series 'df3'
df3 = df3.str.replace('de Vídeo', '')
df3 = df3.str.replace('De Vídeo', '')
df3 = df3.str.replace('De Video', '')

# Remove espaços em branco no início e no fim de cada string na Series 'df3'
df3 = df3.str.strip()

# Obtém os valores únicos da Series 'df3'
df_names = df3.unique()

# Limita o comprimento de cada nome a 45 caracteres
df_names = [name[:45] for name in df_names]

### 5.6. Análise Gráfica dos Dados

In [173]:
# Lista para armazenar os gráficos individuais
figs = []

# Itera sobre os modelos únicos
for model in df2['model'].unique():
    fig = go.Figure()
    model_data = df2[df2['model'] == model].copy()
    model_data['date'] = model_data['date'].dt.floor('min')

    for product in model_data['full_name'].unique():
        product_data = model_data[model_data['full_name'] == product]
        fig.add_trace(go.Scatter(x=product_data['date'], y=product_data['price'], mode='lines+markers', name=product))

    fig.update_layout(title=f'Evolução dos Preços - Modelo: {model}',
                      xaxis=dict(side='top'),
                      xaxis_title='Data',
                      yaxis_title='Preço',
                      yaxis_tickprefix='R$ ',
                      margin=dict(l=50, r=50, t=150, b=10),
                      legend=dict(x=0, y=-10, orientation='h', font=dict(size=10), itemwidth=30, xanchor='left'),
                      height=1200
                      )

    figs.append(fig)

# Exibir os gráficos
for i, fig in enumerate(figs, 1):
    fig.show()

## 6. Análise de Regressão e Análises de Dados

### 6.1. Cálculo da diferença de tempo em horas entre as datas

In [174]:
# Converte a coluna 'date' para o tipo de dado datetime
df['date'] = pd.to_datetime(df['date'])

# Obtém a data inicial no DataFrame 'df' e atribui à variável 'data_inicial'
data_inicial = df['date'].min()

# Calcula a diferença de tempo em horas entre cada data na coluna 'date' e a data inicial
# A função 'dt.total_seconds()' retorna o total de segundos entre as duas datas
# Em seguida, divide esse total por 3600 para obter o número de horas
df2['hour'] = (df['date'] - data_inicial).dt.total_seconds() / 3600
df2['hour'] = df2['hour'].round()

### 6.2. Preparar os dados de entrada e saída

In [175]:
# Faz uma cópia do DataFrame 'df2' e atribui à variável 'df_produtos'
df_produtos = df2.copy()

# Remove as colunas 'full_name', 'link', 'sku', 'weight' e 'date' do DataFrame 'df_produtos'
# O parâmetro axis=1 indica que as colunas devem ser removidas
df_produtos = df_produtos.drop(['full_name', 'link', 'sku', 'weight', 'date'], axis=1)

# Transforma as variáveis categóricas 'brand_name' e 'model' em variáveis dummy
# Isso cria variáveis binárias para cada categoria presente nessas colunas
df_produtos = pd.get_dummies(df_produtos, columns=['brand_name', 'model'], dtype=int, drop_first=True)

# Define os dados de entrada (X) e saída (y)
# X são todas as colunas, exceto a coluna 'price'
# y é a coluna 'price'
X = df_produtos.drop(['price'], axis=1)
y = df_produtos['price']

entrada_X = X
saida_y = y

In [176]:
df_produtos

,price,primePrice,primePriceWithDiscount,oldPrice,oldPrimePrice,priceWithDiscount,discountPercentage,rating,ratingCount,available,...,brand_name_Pny,brand_name_Power Color,brand_name_Powercolor,brand_name_Sapphire,brand_name_Zotac,model_RTX 4070 TI,model_RTX 4080,model_RX 7600 XT,model_RX 7700 XT,model_RX 7800 XT
0,6117.64,5934.11,5043.99,6117.64,6117.64,5199.99,15,4,8,1,...,0,0,0,0,0,1,0,0,0,0
1,6941.16,0.00,0.00,0.00,0.00,5899.99,15,0,0,1,...,0,0,0,0,0,1,0,0,0,0
2,6352.93,0.00,0.00,0.00,0.00,5399.99,15,5,2,1,...,0,0,0,0,0,1,0,0,0,0
3,6117.64,5934.11,5043.99,6117.64,6117.64,5199.99,15,5,9,1,...,0,0,0,0,0,1,0,0,0,0
4,5199.99,0.00,0.00,0.00,0.00,5199.99,0,0,0,1,...,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9629,3647.05,3537.64,3006.99,3647.05,3647.05,3099.99,15,5,1,1,...,0,0,0,0,0,0,0,0,1,0
9630,2705.87,0.00,0.00,0.00,0.00,2299.99,15,5,2,1,...,0,0,1,0,0,0,0,1,0,0
9631,2941.16,0.00,0.00,0.00,0.00,2499.99,15,0,0,1,...,0,0,1,0,0,0,0,1,0,0
9632,3058.81,0.00,0.00,0.00,0.00,2599.99,15,0,0,1,...,0,0,0,0,0,0,0,1,0,0


### 6.3. Treinamento dos dados do modelo fornecido

In [177]:
# Divide os dados em conjuntos de treinamento e teste usando train_test_split
# O parâmetro test_size=0.35 indica que 35% dos dados serão usados como conjunto de teste
# O parâmetro random_state=42 define uma semente para a geração de números aleatórios
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Print das variáveis do conjunto de treinamento
print("Variáveis de Treinamento:")
print("X_train (dados de entrada):")
print(X_train)
print("\ny_train (rótulos de saída):")
print(y_train)

# Print das variáveis do conjunto de teste
print("\nVariáveis de Teste:")
print("X_test (dados de entrada):")
print(X_test)
print("\ny_test (rótulos de saída):")
print(y_test)

Variáveis de Treinamento:
X_train (dados de entrada):
      primePrice  primePriceWithDiscount  oldPrice  oldPrimePrice  \
2499     2694.44                 2425.00   3111.10        3111.10   
7760     9161.10                 8244.99   9444.43        9444.43   
2861     8672.93                 7371.99   8941.16        8941.16   
1870     6618.81                 5625.99   6823.52        6823.52   
4087        0.00                    0.00      0.00           0.00   
...          ...                     ...       ...            ...   
5734        0.00                    0.00      0.00           0.00   
5191        0.00                    0.00      0.00           0.00   
5390        0.00                    0.00      0.00           0.00   
860         0.00                    0.00      0.00           0.00   
7270        0.00                    0.00      0.00           0.00   

      priceWithDiscount  discountPercentage  rating  ratingCount  available  \
2499            2799.99               

### 6.4. Validação cruzada

In [178]:
# Instanciar um modelo de regressão linear
modelo = LinearRegression()

# Definir o número de folds para validação cruzada
num_folds = 5

# Criar um objeto KFold para dividir os dados em folds
# O parâmetro n_splits=num_folds indica o número de folds desejado
# O parâmetro shuffle=True realiza uma mistura aleatória dos dados antes de dividir em folds
# O parâmetro random_state=42 define uma semente para a geração de números aleatórios, garantindo reprodutibilidade
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Realizar validação cruzada e obter os scores de R² para cada fold
scores = cross_val_score(modelo, X, y, cv=kf, scoring='r2')

# Calcular a média dos scores de R²
val_cruzada_mean_r2 = scores.mean()

# Calcular o desvio padrão dos scores de R²
val_cruzada_std_r2 = scores.std()

# Imprimir os resultados da validação cruzada
print("R² médio:", val_cruzada_mean_r2)
print("Desvio padrão dos R²:", val_cruzada_std_r2)

R² médio: 0.9977112903871115
Desvio padrão dos R²: 5.3410618377009936e-05


### 6.5. Regressão Linear

In [179]:
# Instanciar um modelo de regressão linear
modelo_linear = LinearRegression()

# Treinar o modelo de regressão linear com os dados de treinamento
modelo_linear.fit(X_train, y_train)

# Fazer previsões utilizando o modelo de regressão linear nos dados de teste
y_pred_linear = modelo_linear.predict(X_test)

# Calcular o coeficiente de determinação (R²) para o modelo de regressão linear
r2_linear = r2_score(y_test, y_pred_linear)

# Instanciar um modelo de árvore de decisão para regressão
modelo_decision_tree = DecisionTreeRegressor()

# Treinar o modelo de árvore de decisão com os dados de treinamento
modelo_decision_tree.fit(X_train, y_train)

# Fazer previsões utilizando o modelo de árvore de decisão nos dados de teste
y_pred_decision_tree = modelo_decision_tree.predict(X_test)

# Calcular o coeficiente de determinação (R²) para o modelo de árvore de decisão
r2_decision_tree = r2_score(y_test, y_pred_decision_tree)

# Imprimir os resultados dos coeficientes de determinação (R²) para os modelos de regressão linear e árvore de decisão
print(f"Coeficiente de Determinação (R²) - Regressão Linear: {r2_linear}")
print(f"Coeficiente de Determinação (R²) - Árvore de Decisão: {r2_decision_tree}")

Coeficiente de Determinação (R²) - Regressão Linear: 0.9976165967359627
Coeficiente de Determinação (R²) - Árvore de Decisão: 0.9999962256535733


### 6.6. Diferença dos Coeficientes com a Validação Cruzada

In [180]:
# Para a Regressão Linear:
r2_medio_validacao_cruzada = val_cruzada_mean_r2
r2_modelo_regressao_linear = r2_linear
diferenca_regressao_linear = r2_modelo_regressao_linear - r2_medio_validacao_cruzada

# Para a Árvore de Decisão:
r2_medio_validacao_cruzada_arvore = val_cruzada_mean_r2
r2_modelo_arvore_decisao = r2_decision_tree
diferenca_arvore_decisao = r2_modelo_arvore_decisao - r2_medio_validacao_cruzada_arvore


# Gerando o bloco de texto em markdown
markdown_text = f"""
A diferença entre os valores de R² obtidos na validação cruzada e aqueles calculados para os modelos de regressão linear e árvore de decisão indica a capacidade dos modelos em generalizar para novos dados.

Para a Regressão Linear:
- R² médio da validação cruzada: {r2_medio_validacao_cruzada}
- Coeficiente de Determinação (R²) do modelo: {r2_modelo_regressao_linear}
- Diferença: {diferenca_regressao_linear:.15f}

Para a Árvore de Decisão:
- R² médio da validação cruzada: {r2_medio_validacao_cruzada_arvore}
- Coeficiente de Determinação (R²) do modelo: {r2_modelo_arvore_decisao}
- Diferença: {diferenca_arvore_decisao:.15f}

Em ambos os casos, os modelos apresentam R² muito próximos durante a validação cruzada e nos dados de teste. Isso sugere que os modelos têm uma capacidade consistente de generalização para novos dados, indicando um bom desempenho e uma boa capacidade de capturar a variabilidade nos dados. No entanto, é importante observar que a diferença entre os valores de R² obtidos na validação cruzada e nos dados de teste pode fornecer insights sobre o potencial de sobreajuste (overfitting) do modelo, especialmente se essa diferença for significativamente grande.
"""

# Exibindo o bloco de texto em markdown
from IPython.display import Markdown
Markdown(markdown_text)


A diferença entre os valores de R² obtidos na validação cruzada e aqueles calculados para os modelos de regressão linear e árvore de decisão indica a capacidade dos modelos em generalizar para novos dados.

Para a Regressão Linear:
- R² médio da validação cruzada: 0.9977112903871115
- Coeficiente de Determinação (R²) do modelo: 0.9976165967359627
- Diferença: -0.000094693651149

Para a Árvore de Decisão:
- R² médio da validação cruzada: 0.9977112903871115
- Coeficiente de Determinação (R²) do modelo: 0.9999962256535733
- Diferença: 0.002284935266462

Em ambos os casos, os modelos apresentam R² muito próximos durante a validação cruzada e nos dados de teste. Isso sugere que os modelos têm uma capacidade consistente de generalização para novos dados, indicando um bom desempenho e uma boa capacidade de capturar a variabilidade nos dados. No entanto, é importante observar que a diferença entre os valores de R² obtidos na validação cruzada e nos dados de teste pode fornecer insights sobre o potencial de sobreajuste (overfitting) do modelo, especialmente se essa diferença for significativamente grande.


### 6.7. Estudo com GridSearchCV para a busca dos melhores parâmetros

In [181]:
# Gerar dados sintéticos para treinamento e teste
X, y = make_regression(n_samples=100, n_features=10, noise=0.1)

# Definir os parâmetros a serem testados na pesquisa em grade
param_grid = {
    'n_jobs': [1, 2, 3, 4, 5, 6, 7]
}

# Criar uma instância do modelo de regressão linear
model = LinearRegression()

# Criar um objeto GridSearchCV para encontrar os melhores parâmetros
grid_search = GridSearchCV(
    estimator=model,  # Modelo de regressão linear
    param_grid=param_grid,  # Parâmetros a serem testados
    cv=3  # Número de dobras na validação cruzada
)

# Realizar a pesquisa em grade para encontrar os melhores parâmetros
grid_search.fit(X_train, y_train)

# Imprimir os melhores parâmetros encontrados pela pesquisa em grade
print(f'Best parameters: {grid_search.best_params_}')

# Imprimir o melhor escore (média da validação cruzada) alcançado
print(f'Best score: {grid_search.best_score_:.2f}')

# Usar o melhor estimador encontrado para fazer previsões nos dados de teste
y_pred = grid_search.best_estimator_.predict(X_test)

# Calcular o erro quadrático médio entre as previsões e os valores reais
mse = mean_squared_error(y_test, y_pred)
print(f'Mean squared error: {mse:.2f}')

Best parameters: {'n_jobs': 1}
Best score: 1.00
Mean squared error: 16196.16


In [182]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.linear_model import ElasticNet

# Função para encontrar os melhores parâmetros usando Grid Search
def encontrar_melhores_parametros(modelo, parametros, X_train, y_train):
    grid_search = GridSearchCV(modelo, parametros, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    return grid_search.best_params_

# Random Forest Regressor
parametros_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'random_state': [0, 42]
}

melhores_parametros_rf = encontrar_melhores_parametros(RandomForestRegressor(), parametros_rf, X_train, y_train)
modelo_rf = RandomForestRegressor(**melhores_parametros_rf)
modelo_rf.fit(X_train, y_train)  # Ajuste do modelo aos dados de treinamento
y_pred_rf = modelo_rf.predict(X_test)

# Avaliação do desempenho do Random Forest Regressor
print("Random Forest Regressor:")
print("Mean Squared Error:", mean_squared_error(y_test, y_pred_rf))
# Outras métricas de desempenho podem ser adicionadas, dependendo da necessidade

# AdaBoostRegressor
parametros_adaboost = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 1],
    'loss': ['linear', 'square', 'exponential'],
    'random_state': [0, 42]
}

melhores_parametros_adaboost = encontrar_melhores_parametros(AdaBoostRegressor(), parametros_adaboost, X_train, y_train)
modelo_adaboost = AdaBoostRegressor(**melhores_parametros_adaboost)
modelo_adaboost.fit(X_train, y_train)  # Ajuste do modelo aos dados de treinamento
y_pred_adaboost = modelo_adaboost.predict(X_test)

# Avaliação do desempenho do AdaBoostRegressor
print("\nAdaBoostRegressor:")
print("Mean Squared Error:", mean_squared_error(y_test, y_pred_adaboost))

# Gradient Boosting Regressor
parametros_gradientboost = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 1],
    'max_depth': [3, 5, 7],
    'random_state': [0, 42]
}

melhores_parametros_gradientboost = encontrar_melhores_parametros(GradientBoostingRegressor(), parametros_gradientboost, X_train, y_train)
modelo_gradientboost = GradientBoostingRegressor(**melhores_parametros_gradientboost)
modelo_gradientboost.fit(X_train, y_train)  # Ajuste do modelo aos dados de treinamento
y_pred_gradientboost = modelo_gradientboost.predict(X_test)

# Avaliação do desempenho do Gradient Boosting Regressor
print("\nGradient Boosting Regressor:")
print("Mean Squared Error:", mean_squared_error(y_test, y_pred_gradientboost))

Random Forest Regressor:
Mean Squared Error: 48.11865835896571

AdaBoostRegressor:
Mean Squared Error: 50491.243048705495

Gradient Boosting Regressor:
Mean Squared Error: 16.5191507853157


In [183]:
melhores_parametros_gradientboost

{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200, 'random_state': 0}

In [184]:
# Criar uma instância do modelo de regressão linear
model = GradientBoostingRegressor(learning_rate=0.1, max_depth=7, n_estimators=200, random_state=0)

# Ajustar o modelo aos dados de treinamento
model.fit(X_train, y_train)

# Fazer previsões nos dados de teste
y_pred = model.predict(X_test)

# Calcular o erro quadrático médio nos dados de teste
mse = mean_squared_error(y_test, y_pred)
print(f'Mean squared error: {mse:.2f}')

# Calcular o erro quadrático médio nos dados de treinamento
train_error = mean_squared_error(y_train, model.predict(X_train))

# Calcular o erro quadrático médio nos dados de teste
test_error = mean_squared_error(y_test, model.predict(X_test))

# Imprimir os erros de treinamento e teste
print(f"Erro de treinamento: {train_error:.2f}")
print(f"Erro de teste: {test_error:.2f}\n")

Mean squared error: 12.10
Erro de treinamento: 0.00
Erro de teste: 12.10



In [185]:
# Criar uma instância do modelo de regressão linear
model = LinearRegression()

# Ajustar o modelo aos dados de treinamento
model.fit(X_train, y_train)

# Fazer previsões nos dados de teste
y_pred = model.predict(X_test)

# Calcular o erro quadrático médio nos dados de teste
mse = mean_squared_error(y_test, y_pred)
print(f'Mean squared error: {mse:.2f}')

# Calcular o erro quadrático médio nos dados de treinamento
train_error = mean_squared_error(y_train, model.predict(X_train))

# Calcular o erro quadrático médio nos dados de teste
test_error = mean_squared_error(y_test, model.predict(X_test))

# Imprimir os erros de treinamento e teste
print(f"Erro de treinamento: {train_error:.2f}")
print(f"Erro de teste: {test_error:.2f}\n")

Mean squared error: 16196.16
Erro de treinamento: 15991.19
Erro de teste: 16196.16



### 6.8. Análise Gráfica dos estudos de aprendizagem do modelo

In [186]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    # Inicializa uma figura do Plotly
    fig = go.Figure()
    
    # Atualiza o layout da figura com título e rótulos dos eixos
    fig.update_layout(title=title, xaxis_title="Training examples", yaxis_title="Score", yaxis=dict(range=ylim))
    
    # Calcula as curvas de aprendizado usando a função learning_curve
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    
    # Calcula a média e o desvio padrão dos escores de treinamento e teste
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    # Adiciona as curvas de aprendizado à figura
    fig.add_trace(go.Scatter(x=train_sizes, y=train_scores_mean, mode='lines+markers', name='Training score'))
    fig.add_trace(go.Scatter(x=train_sizes, y=test_scores_mean, mode='lines+markers', name='Cross-validation score'))

    # Adiciona as áreas sombreadas representando o intervalo de confiança
    fig.add_trace(go.Scatter(
        x=np.concatenate([train_sizes, train_sizes[::-1]]),
        y=np.concatenate([train_scores_mean - train_scores_std,
                          (train_scores_mean + train_scores_std)[::-1]]),
        fill='toself',
        fillcolor='rgba(255, 0, 0, 0.1)',
        line=dict(color='rgba(255, 0, 0, 0)'),
        name='Training score ± 1 std'
    ))

    fig.add_trace(go.Scatter(
        x=np.concatenate([train_sizes, train_sizes[::-1]]),
        y=np.concatenate([test_scores_mean - test_scores_std,
                          (test_scores_mean + test_scores_std)[::-1]]),
        fill='toself',
        fillcolor='rgba(0, 255, 0, 0.1)',
        line=dict(color='rgba(0, 255, 0, 0)'),
        name='Cross-validation score ± 1 std'
    ))

    fig.show()

# Cria uma figura do Plotly para plotar os resultados da regressão
fig = go.Figure()

# Adiciona os pontos de dados previstos versus reais ao gráfico
fig.add_trace(go.Scatter(x=y_test, y=y_pred, mode='markers', marker=dict(color='blue'), name='Predicted vs. Actual'))

# Adiciona uma linha representando a linha ideal (y = x)
fig.add_trace(go.Scatter(x=[min(y_test), max(y_test)], y=[min(y_test), max(y_test)], mode='lines', line=dict(color='black', dash='dash'), name='Ideal Line'))

# Atualiza o layout do gráfico com título e rótulos dos eixos
fig.update_layout(title='Predicted vs. Actual', xaxis_title='Actual', yaxis_title='Predicted', legend=dict(x=0, y=1))

# Exibe o gráfico
fig.show()

# Define o título para as curvas de aprendizado
title = "Learning Curves"

# Plota as curvas de aprendizado para o modelo
plot_learning_curve(model, title, X_train, y_train, ylim=(0.0, 1.01), cv=None, n_jobs=8)